In [17]:
import polars as pl

df = pl.read_csv(
    "../data/input/train.csv",
    dtypes={"far_price": pl.Float64, "near_price": pl.Float64},
)
df.head()

stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
i64,i64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,str
0,0,0,3.1806e6,1,0.999812,1.3380e7,null,null,0.999812,60651.5,1.000026,8493.03,1.0,-3.029704,0,"""0_0_0"""
1,0,0,166603.91,-1,0.999896,1.6422e6,null,null,0.999896,3233.04,1.00066,20605.09,1.0,-5.519986,0,"""0_0_1"""
2,0,0,302879.87,-1,0.999561,1.8194e6,null,null,0.999403,37956.0,1.000298,18995.0,1.0,-8.38995,0,"""0_0_2"""
3,0,0,1.1918e7,-1,1.000171,1.8390e7,null,null,0.999999,2324.9,1.000214,479032.4,1.0,-4.0102,0,"""0_0_3"""
4,0,0,447549.96,-1,0.999532,1.7861e7,null,null,0.999394,16485.54,1.000016,434.1,1.0,-7.349849,0,"""0_0_4"""


### Closing Auction
- nasdaq市場において、市場終了10分前に行われる取引時間を指す
- この期間中は、注文はすぐには約定せず、一旦プールされClosing Auction終了時に決定された価格により、取引が成立する。
- この価格は、できるだけ多くの取引が成立する価格が選ばれる。
  - 終了5分前時点でのこの価格は、`near_price`として公開される
### データの意味
- https://www.kaggle.com/competitions/optiver-trading-at-the-close/discussion/442994
- `near_price`: 終了5分前時点でのuncross priceのこと
- `reference_price`: best bidとbest askによって決まる値

### データ概要
- stock_idごとに、各日ごとのclosing auction時での価格情報がまとめられている
- 銘柄は200種類
- `date_id`はほとんどの銘柄で481日分あるが、それより少ないものがいくつかある。
- `seconds_in_bucket`はclosing auction開始からの経過秒数を表し、stock_id, date_idごとに値は**一意**で540秒まである（9分間あるということ）
- `time_id`はstock_id内で一意。date_id関係なく一意になっている。
- `near_price`は終了5分前に公開されるので、`seconds_in_bucket < 300`ではnullになる

In [69]:
# stock_id, date_idごとのseconds_in_bucketの重複を確認
for row in df.group_by("stock_id", "date_id").agg(pl.col("seconds_in_bucket").value_counts()).sort("stock_id", "date_id")["seconds_in_bucket"]:
    if len([x for x in row if x["counts"] != 1]) > 0:
        print(row)  # => 出力なし => 重複なし

# stock_idごとのtime_idの重複を確認
for row in df.group_by("stock_id").agg(pl.col("time_id").value_counts()).sort("stock_id")["time_id"]:
    if len([x for x in row if x["counts"] != 1]) > 0:
        print(row)  # => 出力なし => 重複なし

print("stock_id ".ljust(50, "="))
print(df["stock_id"].unique().sort().to_numpy())

print("date_id value_counts ".ljust(50, "="))
display(df.group_by("stock_id").agg(pl.n_unique("date_id"))["date_id"].value_counts())

print("seconds_in_bucket ".ljust(50, "="))
print(df["seconds_in_bucket"].unique().sort().to_numpy())

print("time_id ".ljust(50, "="))
print(df["time_id"].unique().sort().to_numpy())

stock_id =========================================
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199]
date_id value_counts =============================


date_id,counts
u32,u32
290,1
480,2
444,2
411,1
393,1
481,189
300,1
186,1
422,1


seconds_in_bucket ================================
[  0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170
 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350
 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530
 540]
time_id ==========================================
[    0     1     2 ... 26452 26453 26454]
